Research Question: Can student-written RateMyProfessors comments be used to predict numerical professor ratings?

The dataset used in this project will be a publicly available data set from Huggingface, you can access the dataset from this link: https://huggingface.co/datasets/ZephyrUtopia/ratemyprofessors_reviews


Loading the dataset from Hugging Face

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ZephyrUtopia/ratemyprofessors_reviews")
print(dataset)

In [ ]:
import pandas as pd
df = dataset["train"].to_pandas()
print(df.columns)
print(df.shape)

In [ ]:
print(df)
print(df["text"].head())
print(df["rating"].value_counts())